# Linear Elasticity

![](linear_elasticity.svg)

*Figure 1*: Linear elastically deformed 1mm $\times$ 1mm Ferrite logo.



## Implementation

The following code is based on the [Linear Elasticity](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/linear_elasticity/) tutorial from the Ferrite.jl documentation, with some comments removed for brevity.
There are two main modifications:

1. Second-order `Lagrange` shape functions are used for field approximation: `ip = Lagrange{RefTriangle,2}()^2`.
2. Four quadrature points are used to accommodate the second-order shape functions: `qr = QuadratureRule{RefTriangle}(4)`.

In [1]:
using Ferrite, FerriteGmsh, SparseArrays
using Downloads: download

Emod = 200.0e3 # Young's modulus [MPa]
ν = 0.3        # Poisson's ratio [-]

Gmod = Emod / (2(1 + ν))  # Shear modulus
Kmod = Emod / (3(1 - 2ν)) # Bulk modulus

C = gradient(ϵ -> 2 * Gmod * dev(ϵ) + 3 * Kmod * vol(ϵ), zero(SymmetricTensor{2,2}))

function assemble_external_forces!(f_ext, dh, facetset, facetvalues, prescribed_traction)
    # Create a temporary array for the facet's local contributions to the external force vector
    fe_ext = zeros(getnbasefunctions(facetvalues))
    for facet in FacetIterator(dh, facetset)
        # Update the facetvalues to the correct facet number
        reinit!(facetvalues, facet)
        # Reset the temporary array for the next facet
        fill!(fe_ext, 0.0)
        # Access the cell's coordinates
        cell_coordinates = getcoordinates(facet)
        for qp in 1:getnquadpoints(facetvalues)
            # Calculate the global coordinate of the quadrature point.
            x = spatial_coordinate(facetvalues, qp, cell_coordinates)
            tₚ = prescribed_traction(x)
            # Get the integration weight for the current quadrature point.
            dΓ = getdetJdV(facetvalues, qp)
            for i in 1:getnbasefunctions(facetvalues)
                Nᵢ = shape_value(facetvalues, qp, i)
                fe_ext[i] += tₚ ⋅ Nᵢ * dΓ
            end
        end
        # Add the local contributions to the correct indices in the global external force vector
        assemble!(f_ext, celldofs(facet), fe_ext)
    end
    return f_ext
end

function assemble_cell!(ke, cellvalues, C)
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the integration weight for the quadrature point
        dΩ = getdetJdV(cellvalues, q_point)
        for i in 1:getnbasefunctions(cellvalues)
            # Gradient of the test function
            ∇Nᵢ = shape_gradient(cellvalues, q_point, i)
            for j in 1:getnbasefunctions(cellvalues)
                # Symmetric gradient of the trial function
                ∇ˢʸᵐNⱼ = shape_symmetric_gradient(cellvalues, q_point, j)
                ke[i, j] += (∇Nᵢ ⊡ C ⊡ ∇ˢʸᵐNⱼ) * dΩ
            end
        end
    end
    return ke
end

function assemble_global!(K, dh, cellvalues, C)
    # Allocate the element stiffness matrix
    n_basefuncs = getnbasefunctions(cellvalues)
    ke = zeros(n_basefuncs, n_basefuncs)
    # Create an assembler
    assembler = start_assemble(K)
    # Loop over all cells
    for cell in CellIterator(dh)
        # Update the shape function gradients based on the cell coordinates
        reinit!(cellvalues, cell)
        # Reset the element stiffness matrix
        fill!(ke, 0.0)
        # Compute element contribution
        assemble_cell!(ke, cellvalues, C)
        # Assemble ke into K
        assemble!(assembler, celldofs(cell), ke)
    end
    return K
end

function linear_elasticity_2d(C)
    logo_mesh = "logo.geo"
    asset_url = "https://raw.githubusercontent.com/Ferrite-FEM/Ferrite.jl/gh-pages/assets/"
    isfile(logo_mesh) || download(string(asset_url, logo_mesh), logo_mesh)

    grid = togrid(logo_mesh)
    addfacetset!(grid, "top", x -> x[2] ≈ 1.0) # facets for which x[2] ≈ 1.0 for all nodes
    addfacetset!(grid, "left", x -> abs(x[1]) < 1.0e-6)
    addfacetset!(grid, "bottom", x -> abs(x[2]) < 1.0e-6)

    dim = 2
    order = 2 # quadratic interpolation
    ip = Lagrange{RefTriangle,order}()^dim # vector valued interpolation

    qr = QuadratureRule{RefTriangle}(4) # 4 quadrature point
    qr_face = FacetQuadratureRule{RefTriangle}(1)

    cellvalues = CellValues(qr, ip)
    facetvalues = FacetValues(qr_face, ip)

    dh = DofHandler(grid)
    add!(dh, :u, ip)
    close!(dh)

    ch = ConstraintHandler(dh)
    add!(ch, Dirichlet(:u, getfacetset(grid, "bottom"), (x, t) -> 0.0, 2))
    add!(ch, Dirichlet(:u, getfacetset(grid, "left"), (x, t) -> 0.0, 1))
    close!(ch)

    traction(x) = Vec(0.0, 20.0e3 * x[1])

    A = allocate_matrix(dh)
    assemble_global!(A, dh, cellvalues, C)

    b = zeros(ndofs(dh))
    assemble_external_forces!(b, dh, getfacetset(grid, "top"), facetvalues, traction)
    apply!(A, b, ch)

    return A, b, dh, cellvalues, ch
end

linear_elasticity_2d (generic function with 1 method)

### Near Null Space (NNS)

In multigrid methods for problems with vector-valued unknowns, such as linear elasticity,
the near null space represents the low energy mode or the smooth error that needs to be captured
in the coarser grid when using SA-AMG (Smoothed Aggregation Algebraic Multigrid), more on the topic
can be found  in [schroder2010](@citet).

For 2D linear elasticity problems, the rigid body modes are:
1. Translation in the x-direction,
2. Translation in the y-direction,
3. Rotation about the z-axis (i.e., $x_3$): each point (x, y) is mapped to (-y, x).

The function `create_nns` constructs the NNS matrix `B ∈ ℝ^{n × 3}`, where `n` is the number of degrees of freedom (DOFs)
for the case of `p` = 1 (i.e., linear interpolation), because `B` is only relevant for AMG.

In [2]:
function create_nns(dh)
    ##Ndof = ndofs(dh)
    grid = dh.grid
    Ndof = 2 * (grid.nodes |> length) # nns at p = 1 for AMG
    B = zeros(Float64, Ndof, 3)
    B[1:2:end, 1] .= 1 # x - translation
    B[2:2:end, 2] .= 1 # y - translation

    # in-plane rotation (x,y) → (-y,x)
    coords = reduce(hcat, grid.nodes .|> (n -> n.x |> collect))' # convert nodes to 2d array
    y = coords[:, 2]
    x = coords[:, 1]
    B[1:2:end, 3] .= -y
    B[2:2:end, 3] .= x
    return B
end

create_nns (generic function with 1 method)

### Setup the linear elasticity problem
Load `FerriteMultigrid` to access the p-multigrid solver.

In [3]:
using FerriteMultigrid

Construct the linear elasticity problem with 2nd order polynomial shape functions.

In [4]:
A, b, dh, cellvalues, ch = linear_elasticity_2d(C);

Info    : Reading 'logo.geo'...
Info    : Done reading 'logo.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Line)
Info    : [ 50%] Meshing curve 9 (Line)
Info    : [ 60%] Meshing curve 10 (Line)
Info    : [ 60%] Meshing curve 11 (Line)
Info    : [ 70%] Meshing curve 12 (Line)
Info    : [ 70%] Meshing curve 13 (Line)
Info    : [ 80%] Meshing curve 14 (Line)
Info    : [ 80%] Meshing curve 15 (Line)
Info    : [ 90%] Meshing curve 16 (Line)
Info    : [ 90%] Meshing curve 17 (Line)
Info    : [100%] Meshing curve 18 (Line)
Info    : Done meshing 1D (Wall 0.00104736s, CPU 0.001047s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (

Construct the near null space (NNS) matrix

In [5]:
B = create_nns(dh)

208×3 Matrix{Float64}:
 1.0  0.0  -1.0
 0.0  1.0   1.0
 1.0  0.0  -0.379758
 0.0  1.0   1.0
 1.0  0.0  -0.454964
 0.0  1.0   0.801535
 1.0  0.0  -1.0
 0.0  1.0   0.656107
 1.0  0.0  -0.775246
 0.0  1.0   0.600673
 ⋮         
 0.0  1.0   0.567386
 1.0  0.0  -0.487217
 0.0  1.0   0.365932
 1.0  0.0  -0.477795
 0.0  1.0   0.677392
 1.0  0.0  -0.534479
 0.0  1.0   0.52256
 1.0  0.0  -0.640261
 0.0  1.0   0.473388

> **Danger**
>
> Since NNS matrix is only relevant for AMG, and it is not used in the p-multigrid solver, therefore, `B` has to provided using linear field approximation (i.e., `p = 1`) when using AMG as the coarse solver, otherwise (e.g., using `Pinv` as the coarse solver), then we don't have to provide it.

Construct the finite element space $\mathcal{V}_{h,p = 2}$

In [6]:
fe_space = FESpace(dh, cellvalues, ch)

FESpace{Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}, Ferrite.CellValues{Ferrite.FunctionValues{1, Ferrite.VectorizedInterpolation{2, Ferrite.RefTriangle, 2, Ferrite.Lagrange{Ferrite.RefTriangle, 2}}, Matrix{Tensors.Vec{2, Float64}}, Matrix{Tensors.Tensor{2, 2, Float64, 4}}, Matrix{Tensors.Tensor{2, 2, Float64, 4}}, Nothing, Nothing}, Ferrite.GeometryMapping{1, Ferrite.Lagrange{Ferrite.RefTriangle, 1}, Matrix{Float64}, Matrix{Tensors.Vec{2, Float64}}, Nothing}, Ferrite.QuadratureRule{Ferrite.RefTriangle, Vector{Float64}, Vector{Tensors.Vec{2, Float64}}}, Vector{Float64}}, Ferrite.ConstraintHandler{Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}, Float64}}(Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}(Ferrite.SubDofHandler{Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}}[Ferrite.SubDofHandler{Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}}(Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangl

### P-multigrid Configuration

#### 1. Galerkin Coarsening Strategy

In [7]:
config_gal = pmultigrid_config(coarse_strategy = Galerkin())
x_gal, res_gal = solve(A, b,fe_space, config_gal;B = B, log=true, rtol = 1e-10)

([-0.010066052417735043, 0.027797092959891082, -0.01240874414390502, 0.027640763158586292, -0.011854385123482435, 0.03365190209634847, -0.011198274908231555, 0.02783032460427619, -0.012028337125774731, 0.030805535642147414  …  -0.002718167122881313, 0.018354231517249135, -0.006543360153482845, 0.028078201021824636, -0.0025044709085162626, 0.01695653579607672, -0.0032230431473498503, 0.01922965599077098, -0.003825764715161167, 0.020257286724516228], [3980.227721655242, 455.14707317023976, 83.10562617425046, 19.830089514541793, 5.472289795751415, 1.6740301037733865, 0.5591426284601044, 0.20049948415025493, 0.07549461363046683, 0.029266726620580375, 0.011527948122701805, 0.004579028070799461, 0.0018268659786891135, 0.0007305725473577155, 0.0002925414805076431, 0.00011723074201158901, 4.7000082133767856e-5])

#### 2. Rediscretization Coarsening Strategy

In [8]:
# Rediscretization Coarsening Strategy
config_red = pmultigrid_config(coarse_strategy = Rediscretization(LinearElasticityMultigrid(C)))
x_red, res_red = solve(A, b, fe_space, config_red; B = B, log=true, rtol = 1e-10)

([-0.01006605241774941, 0.027797092959884813, -0.012408744143935217, 0.027640763158565375, -0.011854385123504447, 0.0336519020963316, -0.011198274908250067, 0.027830324604262383, -0.012028337125788498, 0.03080553564213324  …  -0.0027181671228574743, 0.01835423151724516, -0.00654336015361581, 0.028078201022162425, -0.0025044709085287322, 0.016956535796113524, -0.0032230431473469004, 0.019229655990823005, -0.003825764715173804, 0.020257286724577155], [3980.227721655242, 457.06819374818394, 83.22542839601425, 19.868450689341543, 5.501109849713366, 1.6926739908520694, 0.5693868243804461, 0.20550627263045834, 0.07776351831307342, 0.03024997140091354, 0.01194347964883253, 0.004752219743190657, 0.0018984979075725849, 0.0007600703281681786, 0.0003046574923322057, 0.0001221995027470287, 4.9035686222845414e-5])

### Test the solution

In [9]:
using Test
@testset "Linear Elasticity Example" begin
    println("Final residual with Galerkin coarsening: ", res_gal[end])
    @test A * x_gal ≈ b
    println("Final residual with Rediscretization coarsening: ", res_red[end])
    @test A * x_red ≈ b
end

Final residual with Galerkin coarsening: 4.7000082133767856e-5
Final residual with Rediscretization coarsening: 4.9035686222845414e-5
Test Summary:             | Pass  Total  Time
Linear Elasticity Example |    2      2  0.5s


Test.DefaultTestSet("Linear Elasticity Example", Any[], 2, false, false, true, 1.758588592574777e9, 1.758588593067483e9, false, "/home/runner/work/FerriteMultigrid.jl/FerriteMultigrid.jl/docs/src/tutorials/linear_elasticity.ipynb")

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*